# Predicting music genre using Convolutional Neural Networks

## Description of the dataset

The dataset we will be using in this notebook comes from a dump of the Free Music Archive (FMA), available [in this Github repository](https://github.com/mdeff/fma). The complete description of this project may be found [in this paper](https://arxiv.org/abs/1612.01840).

To summarise, whole dataset is comprised of 106,574 of Creative Commons-licenced tracks from 16,341 artists and 14,854 albums, arranged in a hierarchical structure of 161 genres. However, in this notebook, we will be focusing on a **smaller subset** of this initial dataset: the dataset we will be focusing on will be comprised of **$8,000$ 30 second clips**. Those clipe belong to 8 top genres, balanced with $1,000$ clips per genre.

Our goal in this notebook will be to classify the genre of an unknown track among the 8 genres that are present in our dataset.

## Downloading the dataset

First of all, let's download the data that we will be using. Since the files are quite heavy (>7 GB in total), we download in the Jupyter Notebook instead of having them directly. We then unzip the content of the files.

The whole dataset is entirely in the cloud, so it removes constraints about data space or anything.

**Note: Please note that the whole process may be a bit long (~15 minutes).**


In [4]:
!curl -O https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
!curl -O https://os.unil.cloud.switch.ch/fma/fma_small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  341M  100  341M    0     0  23.5M      0  0:00:14  0:00:14 --:--:-- 25.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7323M  100 7323M    0     0  25.4M      0  0:04:47  0:04:47 --:--:-- 25.7M


In [0]:
!unzip fma_metadata.zip
!unzip fma_small.zip

First of all, let's download the data that we will be using. Since the files are quite heavy (>7 GB in total), we download in the Jupyter Notebook instead of having them directly.

## Data exploration

In [0]:
import pandas as pd
import numpy as np
import os

The full information about the tracks, including the genre, can be found in the file `tracks.csv`. As for the genre, the according code can be retrieved in the file `genres.csv`.

In [116]:
genres = pd.read_csv('/content/fma_metadata/genres.csv')
tracks = pd.read_csv('/content/fma_metadata/tracks.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [117]:
# The first two lines are a bit buggy, so we remove them 
tracks.columns = ["track_id"] + list(tracks.iloc[0])[1:]
tracks = tracks.drop(tracks.index[[0,1]])
tracks = tracks.reset_index()
tracks[:5]

,index,track_id,comments,date_created,date_released,engineer,favorites,id,information,listens,...,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
0,2,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,...,NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
1,3,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,...,NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
2,4,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,...,NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
3,5,10,0,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,4,6,NaN,47632,...,NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
4,6,20,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,...,NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,NaN,3,NaN,[],Spiritual Level
5,7,26,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,...,NaN,1060,en,Attribution-NonCommercial-NoDerivatives (aka M...,193,NaN,4,NaN,[],Where is your Love?
6,8,30,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,...,NaN,718,en,Attribution-NonCommercial-NoDerivatives (aka M...,612,NaN,5,NaN,[],Too Happy
7,9,46,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,...,NaN,252,en,Attribution-NonCommercial-NoDerivatives (aka M...,171,NaN,8,NaN,[],Yosemite
8,10,48,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,...,NaN,247,en,Attribution-NonCommercial-NoDerivatives (aka M...,173,NaN,9,NaN,[],Light of Light
9,11,134,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,...,NaN,1126,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,943,NaN,5,NaN,[],Street Music


If we take a look at the `tracks.csv` file, we notice that there are many columns that we do not need. In fact, we only need two pieces of information: the **track_id** (in order to know to which song we are referring) and the **corresponding genre**.

In [214]:
# We reduce the dataset in order to only keep the track ID and the genre
reduced_tracks = pd.DataFrame({'track_id': tracks.track_id, 'genres': tracks.genres})
reduced_tracks.track_id = reduced_tracks.track_id.astype(int)
reduced_tracks.set_index('track_id', inplace = True) 
reduced_tracks[:10]

,genres
track_id,
2,[21]
3,[21]
5,[21]
10,[10]
20,"[76, 103]"
26,"[76, 103]"
30,"[76, 103]"
46,"[76, 103]"
48,"[76, 103]"


In [168]:
len(reduced_tracks)

106574

There is another problem: we are working on a **small subset** of an original dataset, while the metadata refers to the **entirety of the original dataset** (which explains why the previous dataset has over $100,000$ rows). For this reason, we will need to remove all the lines that refer to tracks that are not present in our sample.

### Finding the track names

In [0]:
# We iterate over all files in the `fma_small` folder, in order to get their names
file_list = []

for root, dirs, files in os.walk("/content/fma_small"):  
    file_list += [file_name for file_name in files]

file_list = file_list[2:]

In [0]:
# We then remove the '.mp3' extension and take the remaining name
track_list = []

for file_name in file_list:
  track_id = int(file_name.split('.')[0])
  track_list += [track_id]

 As a sanity check, we can see that the number of track names we have is indeed equal to the total number of tracks:

In [171]:
len(track_list)

8000

We can then use those track names to reduce the length of our dataset to the expected $8,000$ rows.

In [215]:
reduced_tracks = reduced_tracks.reindex(track_list).sort_index()
reduced_tracks[:10]

,genres
track_id,
2,[21]
5,[21]
10,[10]
140,[17]
141,[17]
148,[1]
182,[27]
190,[17]
193,[17]


### Finding the right genre
The people who have created this dataset have made it so that each song belongs to **one genre** out of **eight possible genres**. However, in this dataset, we can see that some songs have several genres attributed to them – in that case, it means that all its genres are "subsets" of a single "parent" genre.

For most rows, the "parent" genre is the first genre listed (or the only genre mentionned). However, in certain cases, we find that there are **only subgenres** that are present, and that the "parent" genre is missing.

In the end, the goal of this section is to only keep the "parent" genre, which is supposed to be unique, for every track we have in our dataset.

In [0]:
genres.top_level = genres.top_level.astype(int)
genres.genre_id = genres.genre_id.astype(int)
genres.set_index('genre_id', inplace = True)

In [0]:
def parent_genre(genre_list):
  
  """
  This is the function we apply on the `genres` column.
  We replace every genre by its parent genre (if it exists, otherwise we change nothing).
  
  Afterwards, for a given row, if all parent genres are the same, then we remove
  the list and keep the first element only.
  """
  
  genre_list = genre_list[1:-1].split(',')
  genre_list = [int(genres.loc[int(x)].top_level) if int(x) in genres.index else int(x) for x in genre_list]
  
  if len(set(genre_list)) == 1:
    return genre_list[0]
  
  else:
    return genre_list



reduced_tracks.genres = reduced_tracks.genres.apply(parent_genre, 1)

In [219]:
reduced_tracks[:10]

,genres
track_id,
2,21
5,21
10,10
140,17
141,17
148,38
182,12
190,17
193,17


Now, we just need to replace the genre code with the proper genre name.

In [221]:
reduced_tracks.genres = reduced_tracks.genres.apply(lambda x: genres.loc[x].title)
reduced_tracks[:10]

,genres
track_id,
2,Hip-Hop
5,Hip-Hop
10,Pop
140,Folk
141,Folk
148,Experimental
182,Rock
190,Folk
193,Folk


We can now save this reduced .csv file.

In [0]:
reduced_tracks.to_csv('/content/small_tracks.csv')